In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"C:\Users\piotr\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [58]:
from sqlalchemy import text, bindparam
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai_gemini import zaladuj_api_i_klienta, instrukcja_tlumacz, instrukcja_redaktor
from moduly.utils import sklej_warunki_w_WHERE
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from google import genai
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [59]:
# chce stworzyc proces, w ktorym bede mogl zatwierdzac tlumaczenia
# potrzebuje do tego misje w podziale na grupy:
#    a. oryinal,
#    b. przetlumaczone,
#    c. zredagowane

# oraz potrzebuje do tego wypowiedzi w podziale na grupy:
#    a. oryinal,
#    b. przetlumaczone,
#    c. zredagowane

# oraz tlumaczenia NPCow, Slow kluczowych
# oraz sprawdzic kazde slowo w misji/wypowiedzi czy wystepuje w Slowach kluczowych (cross check)

# calosc potrzebuje wyrzucic do excela w takim ukladzie:
#    a. 

In [60]:
silnik = utworz_engine_do_db()

In [67]:
def stworz_excele_do_zatwaierdzenia_tlumaczen(silnik, kraina = None, fabula = None, dodatek = None):

    warunki_sql = sklej_warunki_w_WHERE(kraina, fabula, dodatek)

    q_select_misje = text(f"""
        SELECT m.MISJA_ID_MOJE_PK
        FROM dbo.MISJE AS m
        WHERE 1=1
        AND m.MISJA_ID_MOJE_PK <> 123456789
                        
        {warunki_sql}

        ORDER BY m.MISJA_ID_MOJE_PK ASC
        ;
    """)

    q_select_tytul = text(f"""
        SELECT MISJA_ID_MOJE_PK, MISJA_TYTUL_EN, MISJA_TYTUL_PL, NAZWA_LINII_FABULARNEJ_EN, NAZWA_LINII_FABULARNEJ_PL
        FROM dbo.MISJE
        WHERE MISJA_ID_MOJE_PK IN :misje_id
    ;
    """).bindparams(bindparam("misje_id", expanding=True))

    parametry = {
        "kraina_en": kraina, 
        "fabula_en": fabula, 
        "dodatek_en": dodatek
    }

    with silnik.connect() as conn:
        misje = conn.execute(q_select_misje, parametry).scalars().all()
        tytuly = conn.execute(q_select_tytul, {"misje_id": misje}).mappings().all()
        return tytuly


In [68]:
stworz_excele_do_zatwaierdzenia_tlumaczen(silnik, kraina="Exile's Reach")

[{'MISJA_ID_MOJE_PK': 2, 'MISJA_TYTUL_EN': "A Druid's Form", 'MISJA_TYTUL_PL': 'Druidzka postać', 'NAZWA_LINII_FABULARNEJ_EN': 'A Dark Ritual', 'NAZWA_LINII_FABULARNEJ_PL': None},
 {'MISJA_ID_MOJE_PK': 3, 'MISJA_TYTUL_EN': "A Druid's Form", 'MISJA_TYTUL_PL': 'Druidzka postać', 'NAZWA_LINII_FABULARNEJ_EN': 'A Dark Ritual', 'NAZWA_LINII_FABULARNEJ_PL': None},
 {'MISJA_ID_MOJE_PK': 5, 'MISJA_TYTUL_EN': "A Hunter's Trap", 'MISJA_TYTUL_PL': 'Pułapka łowcy', 'NAZWA_LINII_FABULARNEJ_EN': 'A Dark Ritual', 'NAZWA_LINII_FABULARNEJ_PL': None},
 {'MISJA_ID_MOJE_PK': 6, 'MISJA_TYTUL_EN': "A Hunter's Trap", 'MISJA_TYTUL_PL': 'Pułapka Łowcy', 'NAZWA_LINII_FABULARNEJ_EN': 'A Dark Ritual', 'NAZWA_LINII_FABULARNEJ_PL': None},
 {'MISJA_ID_MOJE_PK': 8, 'MISJA_TYTUL_EN': "A Mage's Knowledge", 'MISJA_TYTUL_PL': 'Wiedza maga', 'NAZWA_LINII_FABULARNEJ_EN': 'A Dark Ritual', 'NAZWA_LINII_FABULARNEJ_PL': None},
 {'MISJA_ID_MOJE_PK': 9, 'MISJA_TYTUL_EN': "A Mage's Knowledge", 'MISJA_TYTUL_PL': 'Wiedza maga', 'NAZ